> This notebook trains an optimal policy using spinup PPO() function for the LunarLander task. 

> Here is another [experiment](https://colab.research.google.com/drive/1i0St4Z20e3VqdTKD6mzO84viK-5Xy-Ij) on CartPole task. 

> Here is the [link](https://colab.research.google.com/drive/18LdlDDT87eb8cCTHZsXyS9ksQPzL3i6H) to the first Python notebook to in this series. It presents the preambles needed for working with Gym at Google CoLab and explores Gym environments and simple policies. 

> Here is the [link](https://colab.research.google.com/drive/1tug_bpg8RwrFOI8C6Ed-zo0OgD3yfnWy) to the second Python notebook in this series. It shows how to solve some tasks with random action, assigned deterministic action, or heuristic action and how to render the process of tasks to video.

# CoLab Preambles

Most of the requirements of python packages are already fulfilled on CoLab. To run Gym, you have to install prerequisites like xvbf,opengl & other python-dev packages using the following codes.

[](To be done next time: )
[](https://becominghuman.ai/lets-build-an-atari-ai-part-1-dqn-df57e8ff3b26)

In [3]:
!apt-get update --fix-missing
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y

# Special gym environment
!pip install gym[atari]

# For rendering environment, you can use pyvirtualdisplay.
!pip install pyvirtualdisplay
!pip install piglet
!apt-get install x11-utils
# To activate virtual display 
# need to run a script once for training an agent as follows
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

#
# Import libraries
#
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-

In [9]:
# Install spinningup on CoLab
!pip install git+https://github.com/openai/spinningup.git
!git clone https://github.com/openai/spinningup.git
%cd spinningup


  Cloning https://github.com/openai/spinningup.git to /tmp/pip-req-build-ed1l_flg
  Running command git clone -q https://github.com/openai/spinningup.git /tmp/pip-req-build-ed1l_flg
  Created wheel for spinup: filename=spinup-0.2.0-cp36-none-any.whl size=2027 sha256=f700a4c532dcdd1eae9ff7edef4395448a6818c427bbe1361a67efc0b40fbb00
  Stored in directory: /tmp/pip-ephem-wheel-cache-dyddadml/wheels/71/2f/61/753fd3ab792c488f17c5116c8a529f3d6c9447b239fab52570
Successfully built spinup
Cloning into 'spinningup'...
remote: Enumerating objects: 1263, done.
remote: Total 1263 (delta 0), reused 0 (delta 0), pack-reused 1263
Receiving objects: 100% (1263/1263), 31.02 MiB | 34.64 MiB/s, done.
Resolving deltas: 100% (591/591), done.
/content/spinningup


In [10]:
!ls -lqr

total 32
-rwxr-xr-x 1 root root 1155 Mar  1 11:47 travis_setup.sh
drwxr-xr-x 2 root root 4096 Mar  1 11:47 test
drwxr-xr-x 6 root root 4096 Mar  1 11:47 spinup
-rw-r--r-- 1 root root  934 Mar  1 11:47 setup.py
-rw-r--r-- 1 root root   50 Mar  1 11:47 readthedocs.yml
-rw-r--r-- 1 root root 1583 Mar  1 11:47 readme.md
-rw-r--r-- 1 root root 1087 Mar  1 11:47 LICENSE
drwxr-xr-x 9 root root 4096 Mar  1 11:47 docs


# OpenAI Gym

OpenAI gym is a python library that wraps many classical decision problems including robot control, videogames and board games. We will use the environments it provides to test our algorithms on interesting decision problems .

## [Code Format](https://spinningup.openai.com/en/latest/user/algorithms.html#code-format)

All implementations in Spinning Up adhere to a standard template. They are split into two files: 
- An algorithm file, which contains the core logic of the algorithm. 
  - A class definition for an experience buffer object (for storing information from agent-environment interactions).
  - A single function which runs the algorithm. (core)
    - Logger setup
    - Random seed setting
    - Environment inistantiation
    - Making placehodlers from the computation graph
    - Building the actor-critic computation graph via the actor_critic function passed to the algorithm function as an argument
    - Instantiating the experience buffer
    - Building the computation graph for loss functions and diagnostics specific to the algorithm
    - Making training ops
    - Making the TF Session and initiali
    - Setting up model saving through the logger
    - Defining functions needed for running the main loop of the algorithm (eg the core update function, get action function, and test agent function, depending on the algorithm)
    - Running the main loop of the algorithm:
      - Run the agent in the environment
      - Periodically update the parameters of the agent according to the main equations of the algorithm
      - Log key performance metrics and save agent
  - Some support for directly running the algorithm in Gym environment from the common line. 

- A core file, which contains various utilities needed to run the algorithm.
  - Functions related to making and managing placeholders
  - Functions for building sections of computation graph relevant to the actor_critic method for a particular algorithm 
  - Any other useful functions
  - Implementations for an MLP actor-critic compatible with the algorithm, where both the policy and the value function(s) are represented by simple MLPs


## [Running Experiments](https://spinningup.openai.com/en/latest/user/running.html)

- Launching from the Command Line
- Launching from Scripts

Here we focus on **launching from scripts**. 

## [Using ExperimentGride](https://spinningup.openai.com/en/latest/user/running.html#using-experimentgrid)

It’s often useful in machine learning research to run the same algorithm with many possible hyperparameters. Spinning Up ships with a simple tool for facilitating this, called ExperimentGrid.

See this [page](https://spinningup.openai.com/en/latest/user/running.html#using-experimentgrid) for more details. 

## Example: LunarLander-v2

Train PPO in spinup and show resulting policy in a video. 

### Train PPO in spinup

In [0]:
# load packages
import gym
from spinup import ppo_tf1 as ppo
import tensorflow as tf

# after training, load policy and show results in video
from spinup.utils.test_policy import load_tf_policy, run_policy

# train policy
env_fn = lambda : gym.make('LunarLander-v2')

ac_kwargs = dict(hidden_sizes=[64,64], activation=tf.nn.relu)

logger_kwargs = dict(output_dir='path/to/output_dir', exp_name='experiment_name')




In [12]:
ppo(env_fn=env_fn, ac_kwargs=ac_kwargs, steps_per_epoch=5000, epochs=250, logger_kwargs=logger_kwargs)

# Show policy
_, get_action = load_policy('path/to/output_dir')
env2 = gym.make('LunarLander-v2')
env3 = wrap_env(env2)
run_policy(env3, get_action, max_ep_len=500, num_episodes=10)
env3.close()
show_video() 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
|             Epoch |              34 |
|      AverageEpRet |             162 |
|          StdEpRet |            71.3 |
|          MaxEpRet |             220 |
|          MinEpRet |           -4.74 |
|             EpLen |             712 |
|      AverageVVals |            13.6 |
|          StdVVals |            19.1 |
|          MaxVVals |            66.4 |
|          MinVVals |           -14.3 |
| TotalEnvInteracts |        1.75e+05 |
|            LossPi |        9.35e-09 |
|             LossV |             143 |
|       DeltaLossPi |        -0.00986 |
|        DeltaLossV |           -84.8 |
|           Entropy |           0.957 |
|                KL |         0.00691 |
|          ClipFrac |          0.0596 |
|          StopIter |              79 |
|              Time |             349 |
---------------------------------------
---------------------------------------
|             Epoch |              35 |
|      AverageEpRet |             124 |
|   